In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
!apt-get install -y python-numpy python-dev cmake zlib1g-dev libjpeg-dev xvfb libav-tools xorg-dev python-opengl libboost-all-dev libsdl2-dev swig

In [0]:
!pip install gym
!pip install keras

In [0]:
import keras
from keras.models import Model
from keras.layers import Dense, Input
from keras.datasets import mnist
from keras import backend as K

import numpy as np
import random
import matplotlib.pyplot as plt

%matplotlib inline

In [0]:
(X_train, _),(X_test,_)= mnist.load_data()
X_train = X_train.reshape(-1,784).astype('float32')/255.0
X_test = X_test.reshape(-1,784).astype('float32')/255.0


In [0]:
def show_image(pixels):
  img = pixels.reshape(28,28)
  
  plt.axis('off')
  plt.imshow(img, cmap='gray_r')

In [0]:
input_layer= Input(shape=(784,))

hidden1= Dense(512,activation='relu')(input_layer) # Kendinden önceki katmanı giriş olarak ver "input_layer kısmı"
hidden2= Dense(256,activation='relu')(hidden1)
hidden3 =Dense(128,activation='relu')(hidden2)

hidden4 =Dense(64,activation='relu')(hidden3)

hidden5 =Dense(128,activation='relu')(hidden4)
hidden6 =Dense(256,activation='relu')(hidden5)
hidden7 =Dense(512,activation='relu')(hidden6)

decoder= Dense(784,activation='sigmoid')(hidden7)

encoder= K.function([input_layer],[hidden4])


model = Model(input_layer,decoder)
model.compile(optimizer=keras.optimizers.Adadelta(),loss='binary_crossentropy')





In [0]:
model.fit(X_train, X_train, batch_size=128, epochs=10,validation_data=(X_test, X_test))

In [0]:
model.evaluate(X_test,X_test)

In [0]:
sample=random.choice(X_test)

show_image(sample)


In [0]:
reconstucted=model.predict(sample.reshape(1,-1))

show_image(reconstucted)

In [0]:
values=encoder([sample.reshape(1,-1)])[0]

values= (values / np.max(values)).reshape(8,8)

plt.imshow(values, cmap='gray_r')